
## Notebook 08: Validación Cruzada (K-Fold)

 **Objetivo**: Implementar K-Fold Cross-Validation con Regresión Logística para  asegurar que nuestro modelo del Top 25% de contratos sea estadísticamente robusto.



In [10]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.sql.functions import col, when

# %%
spark = SparkSession.builder \
    .appName("SECOP_CrossValidation_Logistica") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

### PASO 0: Preparación de Datos (Binarización al Percentil 75)
Replicamos la lógica del Notebook 07 para tener etiquetas 0 y 1.


In [12]:
df_raw = spark.read.parquet("/opt/spark-data/processed/secop_ml_ready.parquet")

# Crear etiquetas binarias (Top 25%)
discretizer = QuantileDiscretizer(numBuckets=4, inputCol="label", outputCol="cuartil")
df_cuartiles = discretizer.fit(df_raw).transform(df_raw)

df_final = df_cuartiles.withColumn("label_bin", when(col("cuartil") == 3.0, 1.0).otherwise(0.0)) \
                       .select("features", col("label_bin").alias("label"))

train, test = df_final.randomSplit([0.8, 0.2], seed=42)



### RETO 1: Entender K-Fold Cross-Validation
 **Preguntas conceptuales (K=5)**:
1. **Subconjuntos**: Los datos de train se dividen en 5 subconjuntos (folds).
 2. **Modelos entrenados**: Se entrenan 5 modelos por cada combinación de parámetros.
 3. **Porcentaje validación**: En cada iteración se usa el 20% para validar y 80% para entrenar.
4. **Métrica final**: El promedio de las métricas (AUC) de los 5 experimentos.

**Ventaja**: Evita que los resultados dependan de una división "con suerte" de los datos.


## RETO 2: Crear el Modelo Base y Evaluador


In [14]:
# Modelo base sin parámetros fijos (los buscaremos en el Grid)
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=50)

# Evaluador para clasificación binaria
evaluator = BinaryClassificationEvaluator(
    labelCol="label", 
    metricName="areaUnderROC" # AUC es la métrica estándar
)



## RETO 3: Construir el ParamGrid

**Pregunta**: Si probamos 2 valores de regParam y 2 de elasticNetParam con K=3, 
entrenaremos (2x2) * 3 = 12 modelos.


In [15]:
param_grid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1]) \
    .addGrid(lr.elasticNetParam, [0.0, 1.0]) \
    .build()

print(f"Combinaciones en el grid: {len(param_grid)}")


Combinaciones en el grid: 4



## RETO 4: Configurar CrossValidator

**Elección de K**: Usaremos **K=3** para este ejercicio debido al volumen de datos  del SECOP, buscando eficiencia sin perder robustez.


In [20]:
crossval = CrossValidator(
    estimator=lr,
    estimatorParamMaps=param_grid,
    evaluator=evaluator,
    numFolds=3,
    seed=42
)

print(f"Configurado: Cross-Validation con K=3 folds")


Configurado: Cross-Validation con K=3 folds


## RETO 5: Ejecutar Cross-Validation y Analizar


In [21]:
print("Entrenando con Cross-Validation (esto puede tardar unos minutos)...")
cv_model = crossval.fit(train)

# Analizar métricas promedio
avg_metrics = cv_model.avgMetrics
best_metric_idx = avg_metrics.index(max(avg_metrics)) # En AUC buscamos el MÁXIMO

print("\n=== MÉTRICAS PROMEDIO (AUC) POR CONFIGURACIÓN ===")
for i, metric in enumerate(avg_metrics):
    params = param_grid[i]
    reg = params.get(lr.regParam)
    elastic = params.get(lr.elasticNetParam)
    marker = " <-- MEJOR" if i == best_metric_idx else ""
    print(f"Config {i+1}: λ={reg:.2f}, α={elastic:.1f} -> AUC={metric:.4f}{marker}")

# %%
# Evaluar el mejor modelo en el set de Test (datos nunca vistos)
best_model = cv_model.bestModel
predictions = best_model.transform(test)
auc_test = evaluator.evaluate(predictions)

print("\n=== RENDIMIENTO FINAL DEL MEJOR MODELO ===")
print(f"AUC promedio en CV: {max(avg_metrics):.4f}")
print(f"AUC final en Test:  {auc_test:.4f}")

Entrenando con Cross-Validation (esto puede tardar unos minutos)...



=== MÉTRICAS PROMEDIO (AUC) POR CONFIGURACIÓN ===
Config 1: λ=0.01, α=0.0 -> AUC=0.8270 <-- MEJOR
Config 2: λ=0.01, α=1.0 -> AUC=0.8218
Config 3: λ=0.10, α=0.0 -> AUC=0.8263
Config 4: λ=0.10, α=1.0 -> AUC=0.7893

=== RENDIMIENTO FINAL DEL MEJOR MODELO ===
AUC promedio en CV: 0.8270
AUC final en Test:  0.8262


## RETO 6: Comparar CV vs Simple Split

**Confiabilidad**: La métrica de CV es más confiable porque representa el comportamiento 
 del modelo en diferentes "escenarios" de los datos, reduciendo el sesgo.

In [22]:
# Entrenamiento simple con los mismos parámetros
lr_simple = LogisticRegression(
    featuresCol="features", labelCol="label",
    regParam=best_model.getRegParam(),
    elasticNetParam=best_model.getElasticNetParam()
)
model_simple = lr_simple.fit(train)
auc_simple = evaluator.evaluate(model_simple.transform(test))

print(f"AUC con CV:      {auc_test:.4f}")
print(f"AUC sin CV:      {auc_simple:.4f}")
print(f"Diferencia:      {abs(auc_test - auc_simple):.6f}")

# ## Preguntas de Reflexión
#
# 1. **¿K=3 vs K=10?**: K=3 para datasets grandes (ahorro de tiempo); K=10 para datasets pequeños (máxima robustez).
# 2. **¿CV reemplaza el Test Set?**: NO. El test set es la única prueba "ciega" final. CV se usa para elegir el mejor modelo dentro del entrenamiento.
# 3. **¿Dataset de 100 registros?**: Usaría K=10 o incluso Leave-One-Out CV para aprovechar cada dato.
# 4. **¿Time Series?**: No se puede usar K-Fold aleatorio. Se debe usar "TimeSeriesSplit" donde el entrenamiento siempre es anterior a la validación.


AUC con CV:      0.8262
AUC sin CV:      0.8263
Diferencia:      0.000042


In [23]:
# Guardar modelo final optimizado
best_model.write().overwrite().save("/opt/spark-data/processed/modelo_final_cv")
print("Modelo optimizado guardado en /opt/spark-data/processed/modelo_final_cv")

# %%
spark.stop()

Modelo optimizado guardado en /opt/spark-data/processed/modelo_final_cv


La diferencia entre el AUC obtenido con Validación Cruzada y el entrenamiento simple es de apenas 0.000042, el modelo siempre identifica los contratos de alto valor con la misma precisión. el mejor modelo fue la Config 1 ($\lambda=0.01, \alpha=0.0$). Esto indica que una regularización suave tipo Ridge es preferible a Lasso ($\alpha=1.0$). Ridge mantiene todas las variables pero reduce su impacto